In [123]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")
import numpy as np
import matplotlib.pyplot as plt

In [124]:
def preprocess(df):
    # Renommer les colonnes pour plus de clarté
    df = df.rename(columns={"preusuel": "prénom", "annais": "année"})

    # Supprimer les lignes avec des prénoms rares ou des années invalides
    df = df[df["prénom"] != "_PRENOMS_RARES"]
    df = df[df["année"] != "XXXX"]

    # Séparer les données en deux DataFrames selon le sexe
    df_boy = df[df["sexe"] == 1]
    df_girl = df[df["sexe"] == 2]
    
    # Supprimer les colonnes non utilisées
    df_boy = df_boy.drop(columns=["sexe", "dpt"])
    df_girl = df_girl.drop(columns=["sexe", "dpt"])

    # Calculer le nombre total d’occurrences de chaque prénom
    df_boy_grp = df_boy.groupby("prénom")["nombre"].sum().reset_index()
    df_girl_grp = df_girl.groupby("prénom")["nombre"].sum().reset_index()

    # Réduire progressivement le nombre de prénoms en supprimant les moins fréquents (jusqu'à 50)
    occ = 10
    while len(df_boy_grp) > 50 or len(df_girl_grp) > 50:
        # Filtrer les prénoms avec le plus d'occurrences
        df_boy_grp = df_boy_grp[df_boy_grp["nombre"] > occ]
        df_girl_grp = df_girl_grp[df_girl_grp["nombre"] > occ]

        # Recalculer les prénoms communs après filtrage
        common_names = list(set(df_boy_grp["prénom"]) & set(df_girl_grp["prénom"]))

        # Garder seulement les prénoms communs dans les groupes
        df_boy_grp = df_boy_grp[df_boy_grp["prénom"].isin(common_names)]
        df_girl_grp = df_girl_grp[df_girl_grp["prénom"].isin(common_names)]

        occ += 10  # Augmenter le seuil pour resserrer encore

    # Filtrer les DataFrames d’origine pour ne garder que les prénoms communs
    df_boy = df_boy[df_boy["prénom"].isin(common_names)]
    df_girl = df_girl[df_girl["prénom"].isin(common_names)]

    # Convertir les années en entiers
    df_boy["année"]  = df_boy["année"].astype(int)
    df_girl["année"] = df_girl["année"].astype(int)

    # Ajouter une colonne "genre" pour les différencier dans les visualisations
    df_boy["genre"]  = "Homme"
    df_girl["genre"] = "Femme"

    # Fusionner les deux DataFrames et agréger les occurrences
    df_all = pd.concat([df_boy, df_girl], ignore_index=True)
    df_all = df_all.groupby(["année", "prénom", "genre"], as_index=False)["nombre"].sum()

    # Générer un index multi-dimensionnel avec toutes les combinaisons possibles année/prénom/genre
    years   = np.unique(df_all["année"])
    genders = ["Homme", "Femme"]
    mi = pd.MultiIndex.from_product([years, common_names, genders], names=["année", "prénom", "genre"])

    # Reindexer le DataFrame pour combler les valeurs manquantes (remplies avec 0)
    df_all = df_all.set_index(["année", "prénom", "genre"]).reindex(mi, fill_value=0).reset_index()

    # Appliquer une symétrie verticale en inversant les valeurs féminines (pour affichage miroir)
    df_all.loc[df_all["genre"] == "Femme", "nombre"] *= -1

    # Calculer la valeur logarithmique signée (log miroir)
    df_all["log_nombre"] = np.log10(df_all["nombre"].abs() + 1) * np.sign(df_all["nombre"])

    return df_all, common_names, years, genders  # Retourner le DataFrame prêt pour Altair et la liste des noms communs, années et genres


In [125]:
# Lecture du dataset
df = pd.read_csv("../datas/dpt2020.csv", sep=";")

# Preprocessing
df_all, common_names, years, genders = preprocess(df)

In [ ]:
# Définition du slider année
year_param = alt.param(name="Année", bind=alt.binding_range(min=years.min(), max=years.max(), step=1), value=1900)

# Calcul de la borne max absolue du log pour fixer l’axe
max_log = df_all["log_nombre"].abs().max()

labelFontSize=16    # taille des étiquettes
titleFontSize=18

# Construiction du Chart
chart = (
    alt.Chart(df_all)
    .mark_bar()
    .encode(
        x=alt.X(
            "prénom:N",
            sort=common_names,
            axis=alt.Axis(labelAngle=45, title="Prénom",
                          labelFontSize=labelFontSize, titleFontSize=titleFontSize)
        ),
        y=alt.Y(
            "log_nombre:Q",
            scale=alt.Scale(domain=[-max_log, max_log]),
            axis=alt.Axis(title="log() (↑Homme, ↓Femme)",
                          labelFontSize=labelFontSize, titleFontSize=titleFontSize)
        ),
        color=alt.Color(
            "genre:N",
            scale=alt.Scale(domain=genders,
                            range=["steelblue","lightpink"]),
            legend=alt.Legend(title="Genre",
                              titleFontSize=titleFontSize, labelFontSize=labelFontSize)
        ),
        tooltip=[
            alt.Tooltip("prénom:N",    title="Prénom"),
            alt.Tooltip("nombre:Q",     title="Nombre brut"),
            alt.Tooltip("log_nombre:Q", title="Log()×signe")
        ]
    )
    .add_params(year_param)
    .transform_filter(alt.datum.année == year_param)
    .properties(
        width=1500,
        height=900,
        title=alt.TitleParams(
            text="Répartition (log) des prénoms par année",
            fontSize=titleFontSize
        )
    )
)

# Affichage du chart
chart

alt.Chart(...)

# Idées : 
- sort (réussir) les prénoms sur l'axe x
- proposer une visulisation de la différence (positive ou négative) de chaque prénom par rapport à l'année précédente
- étendre la proposition précédente à une visualisation sur 5, 10 ou 20 ans